In [ ]:
## For more info 
https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-capture.html 

In [28]:
import boto3
from urllib.parse import urlparse
import json
import numpy as np

s3_client = boto3.client('s3')
bucket='classification-data-set'
batch_input = 's3://{}/test'.format(bucket)
print("batch_input: "+batch_input)

batch_job_name = "image-classification-model-2--2020-10-21-21-45-52" #Model folder in bucket


object_categories = ['Apple___Apple_scab','Apple___Black_rot','Apple___Cedar_apple_rust','Apple___healthy',
                     'Blueberry___healthy','Cherry_(including_sour)___healthy','Cherry_(including_sour)___Powdery_mildew',
                     'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot','Corn_(maize)___Common_rust_','Corn_(maize)___healthy',
                     'Corn_(maize)___Northern_Leaf_Blight','Grape___Black_rot','Grape___Esca_(Black_Measles)','Grape___healthy',
                     'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)','Orange___Haunglongbing_(Citrus_greening)','Peach___Bacterial_spot',
                     'Peach___healthy','Pepper','_bell___Bacterial_spot','Pepper','_bell___healthy','Potato___Early_blight',
                     'Potato___healthy','Potato___Late_blight','Raspberry___healthy','Soybean___healthy','Squash___Powdery_mildew',
                     'Strawberry___healthy','Strawberry___Leaf_scorch','Tomato___Bacterial_spot','Tomato___Early_blight','Tomato___healthy',
                     'Tomato___Late_blight','Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite',
                     'Tomato___Target_Spot','Tomato___Tomato_mosaic_virus','Tomato___Tomato_Yellow_Leaf_Curl_Virus']



batch_input: s3://classification-data-set/test


In [16]:
def list_objects(s3_client, bucket, prefix):
    response = s3_client.list_objects(Bucket=bucket, Prefix=prefix)
    objects = [content['Key'] for content in response['Contents']]
    return objects

def get_label(s3_client, bucket, prefix):
    filename = prefix.split('/')[-1]
    s3_client.download_file(bucket, prefix, filename)
    with open(filename) as f:
        data = json.load(f)
        
        index = np.argmax(data['prediction'])
        probability = data['prediction'][index]
    print("Result: label - " + object_categories[index] + ", probability - " + str(probability))
    return object_categories[index], probability

In [20]:
inputs = list_objects(s3_client, bucket, urlparse(batch_input).path.lstrip('/'))
print("inputs: ")
print(*inputs, sep="\n ")
#print("Sample inputs: " + str(inputs[:2]))

inputs: 
test/
 test/AppleCedarRust1.JPG
 test/AppleScab1.JPG
 test/CornCommonRust1.JPG
 test/PotatoEarlyBlight1.JPG
 test/PotatoHealthy1.JPG
 test/TomatoEarlyBlight1.JPG
 test/TomatoYellowCurlVirus1.JPG
 test/TomatoYellowCurlVirus5.JPG


In [29]:
outputs = list_objects(s3_client, bucket, batch_job_name + "/output")
print("outputs: ")
print(*outputs, sep="\n ")
#print("Sample output: " + str(outputs[:2]))
print("======================\n")


# Check prediction result of the first 2 images
[get_label(s3_client, bucket, prefix) for prefix in outputs[0:10]]

outputs: 
image-classification-model-2--2020-10-21-21-45-52/output/AppleCedarRust1.JPG.out
 image-classification-model-2--2020-10-21-21-45-52/output/AppleScab1.JPG.out
 image-classification-model-2--2020-10-21-21-45-52/output/CornCommonRust1.JPG.out
 image-classification-model-2--2020-10-21-21-45-52/output/PotatoEarlyBlight1.JPG.out
 image-classification-model-2--2020-10-21-21-45-52/output/PotatoHealthy1.JPG.out
 image-classification-model-2--2020-10-21-21-45-52/output/TomatoEarlyBlight1.JPG.out
 image-classification-model-2--2020-10-21-21-45-52/output/TomatoYellowCurlVirus1.JPG.out
 image-classification-model-2--2020-10-21-21-45-52/output/TomatoYellowCurlVirus5.JPG.out

Result: label - Apple___Cedar_apple_rust, probability - 0.9996086955070496
Result: label - Apple___Apple_scab, probability - 0.9981800317764282
Result: label - Corn_(maize)___Common_rust_, probability - 0.9999963045120239
Result: label - Pepper, probability - 0.9999841451644897
Result: label - Potato___Early_blight, pr

[('Apple___Cedar_apple_rust', 0.9996086955070496),
 ('Apple___Apple_scab', 0.9981800317764282),
 ('Corn_(maize)___Common_rust_', 0.9999963045120239),
 ('Pepper', 0.9999841451644897),
 ('Potato___Early_blight', 0.9991016387939453),
 ('Strawberry___Leaf_scorch', 0.9493405818939209),
 ('Tomato___Septoria_leaf_spot', 0.9999526739120483),
 ('Tomato___Septoria_leaf_spot', 0.9999203681945801)]